In [13]:
from tqdm import tqdm
import openai
import os
import re
import csv

def process_vignette_files(input_dir, output_dir, instruction, dry_run=False):
    """
    Copies vignette data files, extracts each [text] section, prepends instructions,
    and saves them to an output directory as individual prompts.

    Args:
        input_dir (str): Directory containing raw vignette data files.
        output_dir (str): Directory to save processed prompts.
        instruction (str): Instruction to prepend to each prompt.
        dry_run (bool): If True, skips writing files to the output directory.

    Returns:
        list: List of processed prompts with metadata (file name, full prompt).
    """
    # Ensure output directory exists
    vignette_prompts_dir = os.path.join(output_dir, "prompts")
    if not dry_run:
        os.makedirs(vignette_prompts_dir, exist_ok=True)

    prompts_with_metadata = []
    num_cases = 0

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".txt"):
            input_file_path = os.path.join(input_dir, file_name)

            # Read raw file content
            with open(input_file_path, "r") as file:
                raw_content = file.read()

            # Extract each [text] section
            # text_sections = re.findall(r"\[text\](.*?)\n(?=\[|\Z)", raw_content, re.DOTALL)
            text_sections = re.findall(r"\[text\]\s*(.*?)(?=\n\[|\Z)", raw_content, re.DOTALL) 
            # Process each [text] section as a separate prompt
            for idx, text_section in enumerate(text_sections):
                text_section = text_section.strip()
                num_cases = num_cases + 1 

                if text_section:
                    
                    # Prepend instructions
                    full_prompt = f"{instruction}\n\n{text_section}"
                    prompt_file_name = f"{os.path.splitext(file_name)[0]}_case_{idx + 1}.txt"
                    prompts_with_metadata.append((prompt_file_name, full_prompt))

                    if not dry_run:
                        # Save processed prompt to output directory
                        output_file_path = os.path.join(vignette_prompts_dir, prompt_file_name)
                        with open(output_file_path, "w") as file:
                            file.write(full_prompt)
                else:
                    raise ValueError(f"Empty [text] section in file: {file_name}")

    return prompts_with_metadata


def generate_completions(prompts_with_metadata, output_dir, model, dry_run=False):
    """
    Calls OpenAI API for each prompt and emits a result file with service answers and metadata.

    Args:
        prompts_with_metadata (list): List of tuples (file_name, full_prompt).
        output_dir (str): Directory to save results.
        model (str): OpenAI model name to use.
        dry_run (bool): If True, skips calling the API and writing result files.
    """
    if not dry_run:
        # Read API key
        api_key_file = os.path.expanduser("~/openai.key")
        with open(api_key_file, "r") as file:
            openai.api_key = file.read().strip()

        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

    result_rows = []  # To store rows for the result file

    for file_name, full_prompt in tqdm(prompts_with_metadata, desc="Processing prompts"):
        if dry_run:
            # Print simulated workflow for dry run
            print(f"Simulated API call for file: {file_name}")
            print(f"Prompt:\n{full_prompt}\n")
            result_rows.append(["Simulated response", file_name])
        else:
            # Call OpenAI API
            messages = [{'role': "user", 'content': full_prompt}]
            response = openai.chat.completions.create(model=model, messages=messages)

            # Extract generated text
            generated_text = response.choices[0].message.content
            result_rows.append([generated_text, file_name])

    if not dry_run:
        # Emit result file
        result_file_path = os.path.join(output_dir, "result.csv")
        with open(result_file_path, "w", newline="") as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["service_answer", "metadata"])  # Header row
            csv_writer.writerows(result_rows)

        print(f"Processing completed. Results saved to {result_file_path}")
    else:
        print("Dry run completed. No files written or API calls made.")


# Directory paths
input_directory = "../supplemental_data/gpt_o1_preview_VIGNETTE/raw_vignettes"
output_directory = "../supplemental_data/gpt_o1_preview_VIGNETTE"

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Instructions to prepend
instruction = (
    "I am running an experiment on a clinical case report to see how your diagnoses compare with those of human experts.\n"
    "I am going to give you part of a medical case. In this case, you are “Dr. GPT-4”, an AI language model who is providing\n"
    "a diagnosis. Here are some guidelines. First, there is a single definitive diagnosis, and it is a diagnosis that is known\n"
    "today to exist in humans. The diagnosis is almost always confirmed by some sort of genetic test, though in rare cases\n"
    "when such a test does not exist for a diagnosis the diagnosis can instead be made using validated clinical criteria or\n"
    "very rarely just confirmed by expert opinion. After you read the case, I want you to give a differential diagnosis with\n"
    "a list of candidate diagnoses ranked by probability starting with the most likely candidate. Each candidate should be\n"
    "specified with disease name. For instance, if the first candidate is Branchiooculofacial syndrome and the second is\n"
    "Cystic fibrosis, provide this:\n\n"
    "1. Branchiooculofacial syndrome\n"
    "2. Cystic fibrosis\n\n"
    "This list should provide as many diagnoses as you think are reasonable. You do not need to explain your reasoning,\n"
    "just list the diagnoses. Here is the case:"
)

# OpenAI model name
model_name = "o1-preview-2024-09-12"
dry_run = False

# Process raw vignette data and generate prompts
prompts_with_metadata = process_vignette_files(input_directory, output_directory, instruction, dry_run)

# Generate completions using OpenAI API
generate_completions(prompts_with_metadata, output_directory, model_name, dry_run)

KeyboardInterrupt: 